In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
import re
import pandas as pd
import numpy as np

def extract_flight_data(origins, destinations, startdates):
    flight_data = {
        "Origin": [],
        "Destination": [],
        "StartDate": [],
        "DepartureTime": [],
        "ArrivalTime": [],
        "Price": [],
        "Airline": [],
        "Stops": [],
        "LayoverAirports": [],
        "TravelTime": [],
        "ClassType": []
    }

    # Set Chrome options and the user-agent
    opts = Options()
    opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

    # Initialize the ChromeDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

    for origin in origins:
        for destination in destinations:
            for startdate in startdates:
                url = f"https://www.kayak.cl/flights/{origin}-{destination}/{startdate}?sort=bestflight_a"
                # Open the Kayak website
                driver.get(url)
             

                # Presionar multiples veces 'ver mas resultaods' para obtener la mayor cantidad de vuelos posibles por búsqueda
                while True:
                    try:
                        boton = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//div[@role="button" and @class="ULvh-button show-more-button"]')))
                        boton.click()
                    except:
                        print("Button disappeared or not clickable. Stopping.")
                        break

                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Extraer Horarios de salida y llegada
                deptime = []
                arrtime = []
                div_elements_times = soup.find_all('div', class_='vmXl vmXl-mod-variant-large')

                for div in div_elements_times:
                    spans = div.find_all('span')
                    if len(spans) == 3:
                        deptime.append(spans[0].get_text())
                        arrtime.append(spans[2].get_text())

                # Extraer todos los precios
                prices_int = []
                div_elements_price = soup.find_all('div', class_='f8F1-price-text')

                for div in div_elements_price:
                    price_text = div.get_text()
                    price_text = price_text.replace('$', '').replace('.', '')
                    prices_int.append(int(price_text))

                # Extraer lineas aereas
                lineas = []
                div_elements_lineas = soup.find_all('div', class_='J0g6-operator-text')

                for div in div_elements_lineas:
                    lineas_text = div.get_text()
                    lineas.append(lineas_text)

                # Extraer lineas aereas
                escalas = []
                span_elements_escalas = soup.find_all('span', class_='JWEO-stops-text')

                for div in span_elements_escalas:
                    escalas_text = div.get_text()
                    escalas.append(escalas_text)

                # Extraer aeropuertos de escalas
                cleaned_airport_codes = []
                div_elements_jweo = soup.find_all('div', class_='JWEO')

                for div_jweo in div_elements_jweo:
                    div_airport = div_jweo.find('div', class_='c_cgF c_cgF-mod-variant-full-airport')
                    span_elements = div_airport.find_all('span')
                    airport_codes = [span.get_text() for span in span_elements]
                    cleaned_airport_code = ', '.join(set(filter(None, airport_codes)))
                    cleaned_airport_codes.append(cleaned_airport_code)

                cleaned_airport_codes = [', '.join(set(code.split(', '))) for code in cleaned_airport_codes]

                # Extraer horas del viaje
                horas_viaje = []
                div_elements_horas = soup.find_all('div', class_='xdW8 xdW8-mod-full-airport')

                for div in div_elements_horas:
                    horas_text = div.find('div', class_='vmXl vmXl-mod-variant-default').get_text()
                    horas_viaje.append(horas_text)

                # Extraer tipo de clase
                tipo_clase = []
                div_M_JD = soup.find_all('div', class_='M_JD')

                for div in div_M_JD:
                    div_aC3z_links = div.find('div', class_='aC3z-links')
                    div_aC3z_option = div_aC3z_links.find('div', class_='aC3z-option')
                    div_aC3z_name = div_aC3z_option.find('div', class_='aC3z-name').get_text()
                    tipo_clase.append(div_aC3z_name)

                # Agregar los datos a la estructura de datos
                for i in range(len(deptime)):
                    flight_data["Origin"].append(origin)
                    flight_data["Destination"].append(destination)
                    flight_data["StartDate"].append(startdate)
                    flight_data["DepartureTime"].append(deptime[i])
                    flight_data["ArrivalTime"].append(arrtime[i])
                    flight_data["Price"].append(prices_int[i])
                    flight_data["Airline"].append(lineas[i])
                    flight_data["Stops"].append(escalas[i])
                    flight_data["LayoverAirports"].append(cleaned_airport_codes[i])
                    flight_data["TravelTime"].append(horas_viaje[i])
                    flight_data["ClassType"].append(tipo_clase[i])

    # Cerrar el navegador al finalizar
    driver.quit()

    return flight_data

# Definir las listas de origen, destino y fechas
origins = ['SCL']
destinations = ['ANF', 'ARI', 'BBA', 'CJC', 'CPO', 'MHC']
#  'ZCO',
#  'IQQ',
#  'IPC',
#  'LSC',
#  'ZAL',
#  'PNT',
#  'ZOS',
#  'PMC',
#  'PUQ',
#  'SCL',
#  'CCP']
startdates = [  '2023-12-04',
                '2024-01-09',
                '2024-02-14',
                '2024-03-21',
                '2024-04-26',
                '2024-06-25',
                '2024-07-28']

# Llamar a la función para extraer datos
flight_data = extract_flight_data(origins, destinations, startdates)

# Convertir los datos a un DataFrame de Pandas
df = pd.DataFrame(flight_data)

KeyboardInterrupt: 

In [ ]:
from datetime import datetime, timedelta
import random

# Fecha de inicio
start_date = datetime(2023, 11, 11)

# Generar lista de fechas aleatorias
random_dates = [start_date + timedelta(days=random.randint(0, 365)) for _ in range(7)]

# Imprimir la lista de fechas
for date in random_dates:
    print(date.strftime('%Y-%m-%d'))


2024-07-07
2024-02-05
2024-06-30
2024-07-27
2023-12-17
2024-05-20
2024-10-21


### Agregar a Base de Datos

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Origin           440 non-null    object
 1   Destination      440 non-null    object
 2   StartDate        440 non-null    object
 3   DepartureTime    440 non-null    object
 4   ArrivalTime      440 non-null    object
 5   Price            440 non-null    int64 
 6   Airline          440 non-null    object
 7   Stops            440 non-null    object
 8   LayoverAirports  440 non-null    object
 9   TravelTime       440 non-null    object
 10  ClassType        440 non-null    object
dtypes: int64(1), object(10)
memory usage: 37.9+ KB


In [18]:
import sqlite3

# Conectar a la base de datos SQLite
conn = sqlite3.connect('puppets.db')

# Guardar el DataFrame en una tabla llamada 'personas'
df.to_sql('vuelos', conn, if_exists='append', index=False)

# Cerrar la conexión
conn.close()


#### Contador de filas tabla Database

In [19]:
import sqlite3

# Conectar a la base de datos SQLite
conn = sqlite3.connect('puppets.db')
nombre_de_la_tabla = 'vuelos'  # Reemplaza 'nombre_de_la_tabla' con el nombre de la tabla que deseas revisar
cursor = conn.cursor()
cursor.execute(f"SELECT COUNT(*) FROM {'vuelos'}")
cantidad_de_filas = cursor.fetchone()[0]
print(f"La tabla '{nombre_de_la_tabla}' tiene {cantidad_de_filas} filas.")
conn.close()


La tabla 'vuelos' tiene 5021 filas.
